In [1]:
from __future__ import print_function
from six.moves import range
from PIL import Image

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import os
import time

import sys
sys.path.append('/home/michaelcai/GANtor-Arts-Center/src/code/')

import numpy as np
import torchfile

from miscc.config import cfg, cfg_from_file
from miscc.utils import mkdir_p
from miscc.utils import weights_init
from miscc.utils import save_img_results, save_model
from miscc.utils import KL_loss
from miscc.utils import compute_discriminator_loss, compute_generator_loss

from tensorboard import summary
from tensorboardX import FileWriter

import torchvision.utils as vutils

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device(0)
print(torch.cuda.is_available())

True


In [3]:
cfg_from_file('../cfg/wikiart_s2_v2.yml')

from modelv2 import STAGE1_G, STAGE2_G, STAGE2_D

Stage1_G = STAGE1_G()

netG = STAGE2_G(Stage1_G)
netG.apply(weights_init)

# stage_1_file = '../../../results/wikiart_stageI_2019_05_29_16_16_57/Model/netG_epoch_90.pth'
stage_2_file = '../../../results/wikiart_stageII_2019_06_01_06_15_03/Model/netG_epoch_80.pth'
# stage_2_file = '../../saved_models/netG_epoch_60.pth'
state_dict = torch.load(stage_2_file, map_location=lambda storage, loc: storage)
netG.load_state_dict(state_dict)
print('Load from: ', stage_2_file)

# state_dict = torch.load(stage_2_file,map_location=lambda storage, loc: storage)
# netG.STAGE1_G.load_state_dict(state_dict)
# print('Load from: ', stage_1_file)
# netG.eval()
# netG.eval()

if cfg.CUDA:
    netG.cuda()
print(netG)
print(netG.state_dict().keys())

# netG.train()

with torch.no_grad():
    netG = netG

/home/michaelcai/GANtor-Arts-Center/src/code/miscc/config.py:99: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


Load from:  ../../../results/wikiart_stageII_2019_06_01_06_15_03/Model/netG_epoch_80.pth
STAGE2_G(
  (STAGE1_G): STAGE1_G(
    (fc): Sequential(
      (0): Linear(in_features=110, out_features=24576, bias=False)
      (1): BatchNorm1d(24576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (upsample1): Sequential(
      (0): Upsample(scale_factor=2.0, mode=nearest)
      (1): Conv2d(1536, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.2)
    )
    (upsample2): Sequential(
      (0): Upsample(scale_factor=2.0, mode=nearest)
      (1): Conv2d(768, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.2)
    )
    (upsample3): Sequent

In [4]:
folders = {0: 'abstract_painting', 
           1: 'cityscape', 
           2: 'genre_painting', 
           3: 'illustration', 
           4: 'landscape', 
           5: 'nude_painting', 
           6: 'portrait',
           7: 'religious_painting',
           8: 'sketch_and_study',
           9: 'still_life'}
print(folders)

{0: 'abstract_painting', 1: 'cityscape', 2: 'genre_painting', 3: 'illustration', 4: 'landscape', 5: 'nude_painting', 6: 'portrait', 7: 'religious_painting', 8: 'sketch_and_study', 9: 'still_life'}


In [5]:
# image_dir = './Generated_Images/'

# nz = 100
# batch_size = 16
# embedding_size = 10


# for i in range(0, embedding_size * 5):
#     noise = Variable(torch.FloatTensor(batch_size, nz))
#     noise = noise.cuda()

#     noise.data.normal_(0, 1)
#     text_embeddings = torch.zeros((batch_size, embedding_size))
#     text_embeddings[:, i % 10] = 1

#     inputs = (text_embeddings, noise)

#     lr_fake, fake = nn.parallel.data_parallel(netG, inputs, [0])
#     print(fake.size())

# #     vutils.save_image(
# #                 fake.data, '%s/fake_samples_cat_%i.png' %
# #                 (image_dir, i), normalize=True)
    

In [6]:
# netG.eval()
image_dir = './Generated_Images/'

nz = 100
batch_size = 16
embedding_size = 10


for i in range(0, embedding_size):
    noise = Variable(torch.FloatTensor(batch_size, nz))
    noise = noise.cuda()

    noise.data.normal_(0, 1)
    text_embeddings = torch.zeros((batch_size, embedding_size))
    text_embeddings[:, i] = 1

    inputs = (text_embeddings, noise)

    lr_fake, fake = nn.parallel.data_parallel(netG, inputs, [0])
    print(fake.size())

    vutils.save_image(
                fake.data, '%s/fake_samples_cat_%i.png' %
                (image_dir, i), normalize=True)
    

torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256])


In [8]:
nz = 100
batch_size = 16
embedding_size = 10
num_iters = 300 #43.2k images for style, 48k for genre

for i in range(num_iters):
    print ("Iter num %i"%(i))
    for class_idx in range(embedding_size):
        noise = Variable(torch.FloatTensor(batch_size, nz))
        with torch.no_grad():
                    fixed_noise = \
                        Variable(torch.FloatTensor(batch_size, nz).normal_(0, 1))
        noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

        noise.data.normal_(0, 1)
        text_embeddings = torch.zeros((batch_size, embedding_size)).cuda()
        text_embeddings[:, class_idx] = 1

#         inputs = (text_embeddings, noise)

        lr_fake, fake = netG(text_embeddings, noise)

        for im_idx, im in enumerate(fake.data):
            
            vutils.save_image(
                        im, '%s%s/%i_%i.png' %
                        (image_dir, folders[class_idx], i, im_idx), normalize=True)

Iter num 0
Iter num 1
Iter num 2
Iter num 3
Iter num 4
Iter num 5
Iter num 6
Iter num 7
Iter num 8
Iter num 9
Iter num 10
Iter num 11
Iter num 12
Iter num 13
Iter num 14
Iter num 15
Iter num 16
Iter num 17
Iter num 18
Iter num 19
Iter num 20
Iter num 21
Iter num 22
Iter num 23
Iter num 24
Iter num 25
Iter num 26
Iter num 27
Iter num 28
Iter num 29
Iter num 30
Iter num 31
Iter num 32
Iter num 33
Iter num 34
Iter num 35
Iter num 36
Iter num 37
Iter num 38
Iter num 39
Iter num 40
Iter num 41
Iter num 42
Iter num 43
Iter num 44
Iter num 45
Iter num 46
Iter num 47
Iter num 48
Iter num 49
Iter num 50
Iter num 51
Iter num 52
Iter num 53
Iter num 54
Iter num 55
Iter num 56
Iter num 57
Iter num 58
Iter num 59
Iter num 60
Iter num 61
Iter num 62
Iter num 63
Iter num 64
Iter num 65
Iter num 66
Iter num 67
Iter num 68
Iter num 69
Iter num 70
Iter num 71
Iter num 72
Iter num 73
Iter num 74
Iter num 75
Iter num 76
Iter num 77
Iter num 78
Iter num 79
Iter num 80
Iter num 81
Iter num 82
Iter num 83
It